# Problema de Otimização de Troco

## Solução Recursiva

1. Caso Base: Se pudermos dar um troco usando exatamente uma moeda, então esse é o mínimo.
2. Caso contrário: O mínimo será o menor entre: uma moeda de 1, 5, 10 ou 25 mais o troco do valor original menos 1, 5, 10 ou 25 centavos e assim por diante ...

In [1]:
def rec_mc(coin_values, change): 
    min_coins = change
    if change in coin_values:
        return 1 
    else:
        for i in [c for c in coin_values if c <= change]: 
            num_coins = 1 + rec_mc(coin_values, change-i) 
            if num_coins < min_coins:
                min_coins = num_coins 
    return min_coins

In [2]:
rec_mc([1, 5, 10, 25], 63)

6

## Solução recursiva, usando tabela de busca

In [3]:
def rec_dc(coin_values, change, known_results): 
    min_coins = change
    if change in coin_values:
        known_results[change] = 1
        return 1
    elif known_results[change] > 0:
        return known_results[change] 
    else:
        for i in [c for c in coin_values if c <= change]: 
            num_coins = 1 + rec_dc(coin_values,
                                   change-i,
                                   known_results) 
            if num_coins < min_coins:
                min_coins = num_coins
                known_results[change] = min_coins 
        return min_coins
    

In [5]:
rec_dc([1, 5, 10, 25], 63, [0]*(64))

6

## Solução de Programação Dinâmica

- Para qualquer valor $a$, para cada denominação $d$, verifique as moedas mínimas para o valor (calculado anteriormente) $a-d$
- Sempre podemos ir de $a-d$ para $a$ com mais uma moeda

In [6]:
def dp_make_change(coin_values, change, min_coins): 
    for cents in range(change+1):
        coin_count = cents
        for j in [c for c in coin_values if c <= cents]:
            if min_coins[cents-j] + 1 < coin_count: 
                coin_count = min_coins[cents-j]+1
        min_coins[cents] = coin_count 
    return min_coins[change]

In [7]:
dp_make_change([1, 5, 10, 25], 63, [0]*64)

6

## Solução de programação dinâmica modificada

In [8]:
def dp_make_change_2(coin_values, change, min_coins, coins_used): 
    for cents in range(change+1):
        coin_count = cents
        new_coin = 1
        for j in [c for c in coin_values if c <= cents]:
            if min_coins[cents-j] + 1 < coin_count: 
                coin_count = min_coins[cents-j]+1 
                new_coin = j
        min_coins[cents] = coin_count
        coins_used[cents] = new_coin 
    return min_coins[change]

In [9]:
def print_coins(coins_used, change): 
    coin = change
    coin_dict = {} 
    while coin > 0:
        this_coin = coins_used[coin] 
        print(this_coin)
        coin = coin - this_coin

In [10]:
cl = [1, 5, 10, 25]
coins_used = [0]*64
coin_count = [0]*64
dp_make_change_2(cl, 63, coin_count, coins_used) 
print_coins(coins_used, 63)
print(coins_used) 

1
1
1
10
25
25
[1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 25, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1, 1, 25, 1, 1, 1, 1, 5, 1, 1, 1, 1, 10, 1, 1, 1]


## Solução usando Programação Linear Inteira

In [ ]:
from ortools.linear_solver import pywraplp

solver = pywraplp.Solver.CreateSolver('SCIP')

infinity = solver.infinity()
x = {}
for i in range(5):
    x[i] = solver.IntVar(0, infinity, 'x[%i]' % i)
print('Number of variables =', solver.NumVariables())

solver.Add(x[0]*1 + x[1]*5 + x[2]*10 + x[3]*25 + x[4]*21 == 63)
print('Number of constraints =', solver.NumConstraints())

solver.Minimize(x[0] + x[1] + x[2] + x[3] + x[4])

status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print("O troco para R$ 0,%d necessita de %d moedas:"%( 63, solver.Objective().Value()))
    print('Moeda de  1 centavo  = ', x[0].solution_value())
    print('Moeda de  5 centavos = ', x[1].solution_value())
    print('Moeda de 10 centavos = ', x[2].solution_value())
    print('Moeda de 25 centavos = ', x[3].solution_value())
    print('Moeda de 21 centavos = ', x[4].solution_value())    
else:
    print("Não foi possível encontrar troco")